In [1]:
import pandas as pd
import numpy as np
import math

raw_data = pd.read_csv("CompleteDataSet_training_competition_1500.csv", sep=",")

data = raw_data[['WristAccelerometer x-axis', 'WristAccelerometer y-axis', 'WristAccelerometer z-axis', 'Activity']]

n_step = 1500
n_batch = int(data.shape[0] / n_step)

# variables used in the paper
k = 15
t = 3
K = 300

xyz = 3

data = data.as_matrix()
calculate = np.zeros([n_batch * int(k / t), 12 + 1])

# for each data
for i in range(n_batch):
    r1 = np.zeros([xyz])
    r2 = np.zeros([xyz])
    
    # for each sliding window
    for j in range(int(k / t)):
        calculate[int(k / t) * i + j][12] = data[i * n_step][xyz]
        
        l1 = np.zeros([xyz])
        s1 = np.zeros([xyz])
        for m in range(xyz):
            l1[m] = data[i * n_step + j * K][m]
            s1[m] = pow(data[i * n_step + j * K][m], 2)
        
        l = np.zeros([xyz])
        s = np.zeros([xyz])
        for m in range(K):
            for n in range(xyz):
                l[n] += data[i * n_step + j * K + m][n]
                s[n] += pow(data[i * n_step + j * K + m][n], 2)
        
        for m in range(xyz):
            r1[m] += (l[m] - l1[m])
            r2[m] += (s[m] - s1[m])
        
            if not (j == 0):
                calculate[int(k / t) * i + j][4 * m] = calculate[int(k / t) * i + j - 1][4 * m + 2]
                calculate[int(k / t) * i + j][4 * m + 1] = calculate[int(k / t) * i + j - 1][4 * m + 3]
            calculate[int(k / t) * i + j][4 * m + 2] = 2 * r1[m] / K
            if (r2[m] - (2 * pow(r1[m], 2) / K)) >= 0:
                calculate[int(k / t) * i + j][4 * m + 3] = math.sqrt((r2[m] - (2 * pow(r1[m], 2) / K)) / (K / 2 - 1))
            else:
                calculate[int(k / t) * i + j][4 * m + 3] = 0.0
                
data = pd.DataFrame(calculate)

data.to_csv("./dataset/training_data_LSTM_FS_1_wrist.csv", sep=",", header=False, index=False)

c:\users\a jin cho\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
